## Work
1. 請自行定義一個 loss function, 為 0.3 * focal loss + 0.7 cross-entropy，訓練並比較結果


In [1]:
import os
import keras

# Disable GPU
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.layers import BatchNormalization

"""
建立神經網路，並加入 BN layer
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
## 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 25
BATCH_SIZE = 1024
MOMENTUM = 0.95

In [7]:
import tensorflow as tf
import keras.backend as K
import math

"""Code Here
撰寫一個 loss function, 使其可以結合 focal loss 與 crossentropy loss
"""
def combined_loss(gamma=2., alpha=4., ce_weights=0.7, fcl_weights=0.3):
    gamma = float(gamma)
    alpha = float(alpha)
    def CE_focal_loss(y_true, y_pred):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        """
        epsilon = 1e-8
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)

        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.math.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        
        ce_loss = keras.losses.categorical_crossentropy(y_true, y_pred)
        return (ce_weights*ce_loss) + (fcl_weights*tf.reduce_mean(reduced_fl) )
    return CE_focal_loss

ce_weights_list = [0., 0.3, 0.5, 0.7, 1]

In [8]:
import itertools
results = {}

for i, ce_w in enumerate(ce_weights_list):
    print("Numbers of exp: %i, ce_weight: %.2f" % (i, ce_w))

    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    """Code Here
    將自定義的 loss function 加入模型
    """
    model.compile(loss=combined_loss(), metrics=["accuracy"], optimizer=optimizer)
    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True
             )
    
    # Collect results
    exp_name_tag = ("exp-%s" % (i))
    results[exp_name_tag] = {'train-loss': model.history.history["loss"],
                             'valid-loss': model.history.history["val_loss"],
                             'train-acc': model.history.history["accuracy"],
                             'valid-acc': model.history.history["val_accuracy"]}

Numbers of exp: 0, ce_weight: 0.00
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
batch_normalization_3 (B

50000/50000 [==============================] - 2s 32us/step - loss: 1.7118 - accuracy: 0.5960 - val_loss: 2.2297 - val_accuracy: 0.4810
Epoch 12/25
50000/50000 [==============================] - 2s 30us/step - loss: 1.6561 - accuracy: 0.6060 - val_loss: 2.2234 - val_accuracy: 0.4848
Epoch 13/25
50000/50000 [==============================] - 2s 32us/step - loss: 1.6031 - accuracy: 0.6182 - val_loss: 2.2006 - val_accuracy: 0.4844
Epoch 14/25
50000/50000 [==============================] - 2s 30us/step - loss: 1.5506 - accuracy: 0.6278 - val_loss: 2.2305 - val_accuracy: 0.4792
Epoch 15/25
50000/50000 [==============================] - 2s 30us/step - loss: 1.4974 - accuracy: 0.6424 - val_loss: 2.2597 - val_accuracy: 0.4854
Epoch 16/25
50000/50000 [==============================] - 1s 30us/step - loss: 1.4497 - accuracy: 0.6516 - val_loss: 2.2170 - val_accuracy: 0.4874
Epoch 17/25
50000/50000 [==============================] - 1s 29us/step - loss: 1.4022 - accuracy: 0.6639 - val_loss: 2.2223

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 2s 36us/step - loss: 3.3834 - accuracy: 0.3072 - val_loss: 3.4148 - val_accuracy: 0.2405
Epoch 2/25
50000/50000 [==============================] - 1s 29us/step - loss: 2.5607 - accuracy: 0.4211 - val_loss: 3.0142 - val_accuracy: 0.3189
Epoch 3/25
50000/50000 [==============================] - 1s 29us/step - loss: 2.3571 - accuracy: 0.4637 - val_loss: 2.7359 - val_accuracy: 0.3937
Epoch 4/25
50000/50000 [==============================] - 1s 29us/step - loss: 2.2335 - accuracy: 0.4859 - val_loss: 2.5577 - val_accuracy: 0.4151
Epoch 5/25
50000/50000 [==============================] - 1s 29us/step - loss: 2.1325 - accuracy: 0.5063 - val_loss: 2.4282 - val_accuracy: 0.4462
Epoch 6/25
50000/50000 [==============================] - 1s 29us/step - loss: 2.0528 - accuracy: 0.5229 - val_loss: 2.3669 - val_accuracy: 0.4498
Epoch 7/25
50000/50000 [==============================] - 1s 29us/st

Epoch 21/25
50000/50000 [==============================] - 1s 30us/step - loss: 1.2474 - accuracy: 0.7004 - val_loss: 2.2829 - val_accuracy: 0.4893
Epoch 22/25
50000/50000 [==============================] - 1s 30us/step - loss: 1.1982 - accuracy: 0.7130 - val_loss: 2.2792 - val_accuracy: 0.4879
Epoch 23/25
50000/50000 [==============================] - 1s 30us/step - loss: 1.1563 - accuracy: 0.7240 - val_loss: 2.3090 - val_accuracy: 0.4909
Epoch 24/25
50000/50000 [==============================] - 1s 30us/step - loss: 1.1120 - accuracy: 0.7345 - val_loss: 2.3738 - val_accuracy: 0.4840
Epoch 25/25
50000/50000 [==============================] - 1s 30us/step - loss: 1.0727 - accuracy: 0.7432 - val_loss: 2.3465 - val_accuracy: 0.4830


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as mplcm
import matplotlib.colors as colors
%matplotlib inline
NUM_COLORS = len(results.keys())

cm = plt.get_cmap('gist_rainbow')
cNorm  = colors.Normalize(vmin=0, vmax=NUM_COLORS-1)
scalarMap = mplcm.ScalarMappable(norm=cNorm, cmap=cm)
color_bar = [scalarMap.to_rgba(i) for i in range(NUM_COLORS)]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()